In [1]:
import sys
import os

import ccxt
import backtesting as bt
from backtesting import Backtest, Strategy
import talib
import pandas_ta as ta

import pandas as pd
import numpy as np

In [2]:

def get_data(symbol, timeframe, limit):
    exchange = ccxt.kraken()
    exchange.load_markets()

    data = exchange.fetch_ohlcv('BTC/USDT', timeframe=timeframe, limit=limit)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).rename(columns={
        'timestamp': 'timestamp',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
data = get_data('BTC/USDT', '1h', 1000)

In [4]:
data

,Open,High,Low,Close,Volume
timestamp,,,,,
2024-04-27 03:00:00,62947.0,63136.9,62841.0,63070.6,3.890570
2024-04-27 04:00:00,63049.1,63161.5,62957.3,63013.3,0.562865
2024-04-27 05:00:00,63029.9,63073.0,62857.5,62981.3,0.806780
2024-04-27 06:00:00,62981.2,63110.7,62890.0,63005.0,1.887178
2024-04-27 07:00:00,63006.9,63139.2,62950.6,62965.5,6.454611
...,...,...,...,...,...
2024-05-26 22:00:00,68494.7,68693.6,68481.9,68595.6,3.915592
2024-05-26 23:00:00,68559.7,68559.8,68356.7,68531.7,1.545142
2024-05-27 00:00:00,68503.1,68762.1,68476.7,68756.5,1.062076


# BBands

In [5]:

class BBANDS_strategy_v1(Strategy):
    ############################ Parameters ####################################
    BB_SMA = 20                         # Bollinger bands SMA
    BB_STD = 2                        # Bollinger bands standard deviation
    BB_MAX_BANDWIDTH = 5                # Bollinger bands maximum volatility allowed
    
    min_volatility = 0.3
    max_buy_perc = 0.2
    min_sell_perc = 0.25

    ############################ Utilities ####################################
    # For logging
    def log(self, txt, dt=None):
        dt = dt or self.data.index[-1]
        # print(f'{dt.strftime("%Y-%m-%d %H:%M:%S")}: {txt}')
    
    ############################ Strategy: Calculate the indicators ####################################

    def init(self):
        
        # print(f"BB_SMA: {self.BB_SMA}, BB_STD: {self.BB_STD}, BB_MAX_BANDWIDTH: {self.BB_MAX_BANDWIDTH}")
        # print(f"min_volatility: {self.min_volatility}, max_buy_perc: {self.max_buy_perc}, min_sell_perc: {self.min_sell_perc}")
        
        self.upper = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 2],
                            self.data.df, self.BB_SMA, self.BB_STD, name='upper')
        
        self.lower = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 0],
                            self.data.df, self.BB_SMA, self.BB_STD, name='lower')
        
        self.volatility = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 1],
                                 self.data.df, self.BB_SMA, self.BB_STD, name='volatility', plot=True, overlay=False)

        self.high_limit = self.I(lambda upper, lower: upper + (upper - lower) / 2, self.upper, self.lower, name='high_limit')
        self.low_limit = self.I(lambda upper, lower: lower - (upper - lower) / 2, self.upper, self.lower, name='low_limit')
        self.close_percentage = self.I(lambda close, low_limit, high_limit: np.clip((close - low_limit) / (high_limit - low_limit), 0, 1), self.data.df['Close'], self.low_limit, self.high_limit, name='close_percentage')
        self.volatility_scaled = self.I(lambda volatility: np.clip(volatility / (100 / self.BB_MAX_BANDWIDTH), 0, 1), self.volatility, name='volatility_scaled')
        
        self.buy_signal = self.I(lambda volatility_scaled, close_percentage: (volatility_scaled > self.min_volatility) & (close_percentage < self.max_buy_perc), self.volatility_scaled, self.close_percentage, name='buy_signal')
        self.sell_signal = self.I(lambda close_percentage: (close_percentage > self.min_sell_perc), self.close_percentage, name='sell_signal')
    
    

    def next(self):
        
        self.log(f"Close: {self.data.Close[-1]}, position: {self.position.size}, cash: {self._broker.margin_available}")
        
        if self.position.size == 0:
            if self.buy_signal[-1]:
                self.log("BUY")
                self.buy()
                
        elif self.position.size > 0:
            if self.sell_signal[-1]:
                self.log("SELL")
                self.trades[0].close()

In [6]:
# BBANDS_strategy.BB_SMA = 40
# BBANDS_strategy.BB_STD = 2
# BBANDS_strategy.BB_MAX_BANDWIDTH = 9
# BBANDS_strategy.min_volatility = 0.6
# BBANDS_strategy.max_buy_perc = 0.3
# BBANDS_strategy.min_sell_perc = 0.2

bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run()
bt.plot()

Row(id='1805', ...)

In [7]:
strat

Start                     2024-04-27 03:00:00
End                       2024-05-27 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                    5.555556
Equity Final [$]                 98985.110575
Equity Peak [$]                 100285.282525
Return [%]                          -1.014889
Buy & Hold Return [%]                9.625087
Return (Ann.) [%]                  -11.317317
Volatility (Ann.) [%]                2.490301
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.310064
Avg. Drawdown [%]                   -0.655639
Max. Drawdown Duration       18 days 06:00:00
Avg. Drawdown Duration        9 days 02:00:00
# Trades                                   11
Win Rate [%]                        45.454545
Best Trade [%]                       0.360375
Worst Trade [%]                     -0.678263
Avg. Trade [%]                    

## Optimization

In [8]:
import multiprocessing as mp
mp.set_start_method('fork')

In [9]:
bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

stats = bt.optimize(
                    BB_SMA=range(10, 70, 5),
                    BB_STD=range(1, 10, 2),
                    BB_MAX_BANDWIDTH=range(1, 10, 1),
                    min_volatility=list(np.arange(0, 0.5, 0.05)),
                    max_buy_perc=list(np.arange(0.1, 1, 0.05)),
                    min_sell_perc=list(np.arange(0, 0.5, 0.05)),
                    maximize='Return [%]',
                    max_tries=500,
                    random_state=1) 

# pd.DataFrame(stats)
# stats._strategy

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 501 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/11 [00:00<?, ?it/s]

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)),

In [10]:
stats._strategy

<Strategy BBANDS_strategy_v1(BB_SMA=55,BB_STD=3,BB_MAX_BANDWIDTH=2,min_volatility=0.1,max_buy_perc=0.5500000000000002,min_sell_perc=0.45)>

In [11]:
bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run( BB_SMA=55, BB_STD=3, BB_MAX_BANDWIDTH=2, min_volatility=0.1, max_buy_perc=0.55, min_sell_perc=0.45)
bt.plot()

Row(id='3532', ...)

In [12]:
strat

Start                     2024-04-27 03:00:00
End                       2024-05-27 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                       53.75
Equity Final [$]                102991.638575
Equity Peak [$]                  104363.79415
Return [%]                           2.991639
Buy & Hold Return [%]                9.625087
Return (Ann.) [%]                   41.492313
Volatility (Ann.) [%]               25.866259
Sharpe Ratio                          1.60411
Sortino Ratio                         3.44179
Calmar Ratio                          7.62776
Max. Drawdown [%]                   -5.439646
Avg. Drawdown [%]                   -0.750865
Max. Drawdown Duration       15 days 08:00:00
Avg. Drawdown Duration        1 days 15:00:00
# Trades                                   94
Win Rate [%]                        57.446809
Best Trade [%]                       2.009314
Worst Trade [%]                     -5.466062
Avg. Trade [%]                    